# Lab 1: Introduction to Remote Sensing and Google Earth Engine

**Purpose:** The purpose of this lab is to enable you to search, find and visualize remotely sensed imagery in Google Earth Engine.  At completion, you should be able to load imagery, extract image property information, visualize data and work with image collections.


**Prerequisites:** If you are new to Python programming, go through [this guide](https://docs.python.org/3/tutorial/index.html).  Review the [Earth Engine documentation](https://developers.google.com/earth-engine/guides) and the [Python Guide](https://developers.google.com/earth-engine/guides/python_install).


A couple of links that are helpful:


1.   [Earth Engine Documentation](https://developers.google.com/earth-engine)
2.   [Earth Engine API Reference](https://developers.google.com/earth-engine/apidocs)
3.   [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets)
4.   [Earth Engine Best Practices](https://developers.google.com/earth-engine/guides/best_practices)



In [ ]:
# import ee api and geemap package
import ee
import geemap
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Hello, Earth Engine

### Client vs Server side ops

It is important to distinguish Earth Engine objects from other Python objects or that might be in your code. You can manipulate objects on the server by manipulating client-side “proxy” objects in your script. You can recognize a proxy object as anything starting with `ee`. These Earth Engine proxy objects do not contain any actual data and are just handles for objects on the server.

[Client vs Server docs](https://developers.google.com/earth-engine/guides/client_server)

In [ ]:
client_string = "Hello world"
type(client_string)

In [ ]:
server_string = ee.String("Hello world")
type(server_string)

In [ ]:
print(f"Client side string:\n{client_string}\n")
print(f"Server side string:\n{server_string}\n")

Remember, Earth Engine proxy objects do not contain any actual data. Therefore, to get the results from Earth Engine, one would need to use the method `.getInfo()`.

In [ ]:
requested_string = server_string.getInfo()
print(f"Server side string:\n{requested_string}\n")

Be careful with using `.getInfo()` as this is requires EE to compute and transfer the data synchronously to the local machine so it can cause slow-downs for your process. This also defeats the purpose of cloud-based computing. However, sometimes `.getInfo()` cannot be avoided so use with caution.

## Loading Earth Engine Data

Earth Engine has a data catalog of popular remote sensing data that one can load, explore the structure and process data. Here was are going to load in an elevation dataset and do some processing.

In [ ]:
# load the NASA DEM elevation dataset
dem = ee.Image("NASA/NASADEM_HGT/001")

### Properties of Images

In [ ]:
# properties of an image
dem.getInfo()

In [ ]:
# get the metadata property from an image
dem.get("tags").getInfo()

In [ ]:
# print the band information
dem.bandNames().getInfo()

In [ ]:
# get spatial metadata from image
proj = dem.projection()

# from this we can extract different information
resolution = proj.nominalScale()
geotransform = proj.transform()

# print the spatial information
print(f"SRTM resolution: {resolution.getInfo()}")
print(f"SRTM GeoTransform: {geotransform.getInfo()}")

### A simple processing example

To test processing some data we will define a define a point (i.e. geometry) for Mount Everest and request the elevation:

In [ ]:
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

### Visualizing Earth Engine Images

To visualize the DEM data we will use `geemap` to define an interactive map and add the `ee.Image` object with visulization parameters:

In [ ]:
# Display result
Map = geemap.Map(zoom=3)

Map.addLayer(dem, {"min":0,"max":3000, "bands":"elevation", "palette":cmaps.get_palette("terrain")}, "DEM")

Map.addLayerControl()

Map

In [ ]:
fc = ee.FeatureCollection(Map.draw_features)

In [ ]:
dem.reduceRegions(
    collection=fc,
    scale=120,
    reducer= ee.Reducer.mean()
).getInfo()

## Image Collections: An Organized Set of Images

You saw some of the basic ways to interact with an individual `ee.Image ` , however, depending on how long a remote-sensing platform has been in operation, there may be thousands or millions of images collected of Earth. In Earth Engine, these are organized into an *ImageCollection*, a specialized data type that has specific operations available in the Earth Engine API.

We work with image collections in complex ways later on. For now, we will view and work with their most basic attributes, and use these skills to view some of the major types of image collections in Earth Engine. We will view some of the different types of data sets in the following sections, including climate and weather data, digital elevation models and other terrain data, land cover, cropland, satellite imagery, and others.



### View an Image Collection

The Landsat program is a joint NASA/USGS program that has launched a sequence of Earth observation satellites, named Landsat 1, 2, etc. Landsats have been returning images since 1972, making that collection of images the longest continuous observation of the Earth's surface. We will now view images and basic information about one of the image collections that is still growing: collections of scenes taken by Landsat 8, launched in 2013.

In [ ]:
# load in the landsat collection
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")

In [ ]:
# print the number of images in a collection
n_imgs = landsat8.size().getInfo()

print(f"The size of the Landsat 8 image collection is: {n_imgs}");

In [ ]:
# Display result
Map = geemap.Map(zoom=3)

Map.addLayer(landsat8, {"bands": ['B4','B3','B2'], "min": 5000, "max": 15000}, 'Landsat 8 Image Collection');

Map.addLayerControl()

Map

Notice the high amount of cloud cover, and the “layered” look. This is because Earth Engine is drawing each of the images that make up the ImageCollection one on top of the other (i.e. mosaic operation). The striped look is the result of how the satellite collects imagery. The overlaps between images and the individual nature of the images mean that these are not quite ready for analysis.


### Filtering Image Collections
The ImageCollection data type in Earth Engine has multiple approaches to filtering which helps to pinpoint the exact images you want to view or analyze from the larger collection.

#### Filter by time

The first type of filtering is by time. We can define a date range for the whole collection.

In [ ]:
# specify the start and end dates of interest
start_time = "2021-01-01"
end_time = "2022-01-01" # end date is exclusive

# apply date range filter
landsat8_filtered = landsat8.filterDate(start_time,end_time)

In [ ]:
# print the number of images in the filtered collection
n_imgs_filtered = landsat8_filtered.size().getInfo()

print(f"The size of the Landsat 8 image collection is: {n_imgs_filtered}")

The number is significantly lower than the number of images in the entire collection. This is the result of filtering the dates to one year (2021).


In [ ]:
# Display result
Map = geemap.Map(zoom=3)

Map.addLayer(landsat8_filtered, {"bands": ['B4','B3','B2'], "min": 5000, "max": 15000}, 'Landsat 8 Image Collection');

Map.addLayerControl()

Map

While this example shows a linear range to filter by time, a calendar range can also be used to filter data for multiple years to a specific season.

#### Filter by space

The next filter we will examine is filtering the collection to a specific location by a geometry.

In [ ]:
# define a point for Provo, UT
provo = ee.Geometry.Point(-111.6585, 40.2338)

In [ ]:
# filter the collection to the geometry
landsat8_provo = landsat8_filtered.filterBounds(provo)

In [ ]:
# Display result
Map = geemap.Map()

Map.centerObject(provo, 10)

Map.addLayer(landsat8_provo, {"bands": ['B4',"B3","B2"], "min": 5000, "max": 15000}, 'Landsat 8 Image Collection');

Map.addLayerControl()

Map

#### Filter by property

The methods `.filterDate()` and `.filterBounds()` we have used are special filters because they are used so often. Underneath, the filtering is applied using a filter object (i.e. `ee.Filter()`) on metadata properties. We we go through an example of how this works to filter imagery with lower cloud cover.

In [ ]:
landsat8.limit(5).getInfo()

In [ ]:
landsat8.first().getInfo()

In [ ]:
# define a filter object
filter_op = ee.Filter.lt("CLOUD_COVER",25)

# apply the filter on a collection
landsat8_noclouds = landsat8_provo.filter(filter_op)

In [ ]:
landsat8_noclouds.first().date().format("YYYY-MM-dd HH:mm:ss").getInfo()

In [ ]:
ee.Date("2015-01-01")

In [ ]:
# Display result
Map = geemap.Map()

Map.centerObject(provo, 10)

Map.addLayer(landsat8_noclouds.sort("system:time_start"), {"bands": ['B4','B3','B2'], "min": 5000, "max": 15000}, 'Landsat 8 Image Collection');
Map.addLayer(landsat8_noclouds.first(), {"bands": ['B4','B3','B2'], "min": 5000, "max": 15000}, 'Landsat 8 Image');

Map.addLayerControl()

Map

### Sorting

After filtering we can apply additional operations to extract the imagery we would like (or manage data for processing). Here we will sort the imagery by date to extract the earliest image from the filtered collection. Sorting work on any numeric metadata property so it can be very useful to extract imagery or pre process things.

In [ ]:
# sort the collection by a numeric value, in this case time
landsat8_sorted = landsat8_noclouds.sort("CLOUD_COVER")

# grab the first image in the sorted collection
landsat8_img = landsat8_sorted.first()

In [ ]:
# check that the result is in fact an image not a collection
assert type(landsat8_img) == ee.Image

In [ ]:
landsat8_img.date().format("YYYY-MM-dd").getInfo()

In [ ]:
# Display map
Map = geemap.Map()

Map.centerObject(provo, 10)

Map.addLayer(landsat8_img, {"bands": ['B7','B5','B3'], "min": 5000, "max": 15000}, 'Landsat 8 Image Collection');

Map.addLayerControl()

Map

## Conclusion 

In this notebook we explored a couple fundamental concepts of using Earth Engine:
1. Client vs Server side operations
2. Loading data using Earth Engine and exploring the properties of data
3. Visualizing data
4. Loading image collections using Earth Engine
5. Filtering image collections using space, time, and metadata information
6. Combining filtering with sorting to gather imagery we are interested in

These concepts are used often with Earth Engine and should be internalized. The next lab will focus on using the information to explore charateristics of different sensors/data in Earth Engine.